<a href="https://colab.research.google.com/github/cricriii/Projects/blob/main/IFT6575_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transport optimization and scheduling

# Introduction

**Main idea** \\
I will model a problem the population of Montreal faced late 2024. At that time, it was announced by the STM that three stations on the blue line would be close due to some negligeance abount maintenance; the stations in question are Fabre, D'Iberville and St-Michel. As a temporary solution, the STM offered to put in place some shuttle buses. \\

For my project, I will try to create a simplified model of this problem to solve the following questions: \\
- How many shuttle buses should be on the roads depending on the hour or the day?
- If we have a certain bank of employees, how should the employees be scheduled so that we have an employee each time a bus is used?

*Some of the data and values that I will use are real (found online), but for those that I could not find, I will generate them myself.*

# Step 1 : generate random data

First, we need to generate the following data:

Before the shutdown: \\
- Frequency of the metro from Jean-Talon Station to St-Michel Station / hour.
- Amount of passengers in the metro that go form Jean-Talon to St-Michel / hour.

Let's set some variables: \\
**`Hour`**: the hour of the day (from 5:30 AM to 1 AM). \\
**`Metro_Freq_per_Hour`**: the number of metro trips per hour, with higher frequency during rush hours (7-9 AM and 4-6 PM). \\
**`Passengers_per_Hour`**: the number of passengers per hour, with higher numbers during peak hours. \\



Here is public information about the Blue Line I found on STM's website: \\
**`Operation hours:`** \\

| Type       |Towards|Saint-Michel |  \|  |Towards|Snowdon|
| :--------- | :----: | :----: | :------- | :----: | :----: |
|     	     | First  | Last	|  \|   | First | Last  |
| Weekdays   | 05h30	| 00h45	|  \|   | 05h30 |	00h45 |
| Saturday	 | 05h30	| 01h15 |  \|   |	05h30	| 01h15 |
| Sunday	   | 05h30	| 00h45	|  \|   |05h30	| 00h45 |



**`Frequency of departures:`** \\

| Type       |Peak | Non-peak |
| :--------- | :----: | :----: |
|     	     | 7am to 9am and 4pm to 6pm | Morning, day and evening (other hours)|
| Weekdays   | Every 3 to 5 minutes |	Every 4 to 10 minutes |
| Week-end	 | Every 8 to 11 minutes | Every 8 to 11 minutes |


Also, a small problem I was faced with is that I do not know the metro's ridership per day for the Blue Line. My research was unsuccessful and all I could find was the average of 1,029,300 daily unlinked passenger trips per weekday and 2023's report of the numbers of entries in the most busy stations for the year so I decided to do some estimations.

Suppose that the stations on the Blue Line are half as busy as the fifth most busy station, which is Atwater Station with 5 884 878 entries in 2023, so suppose that each station on the Blue Line has about 2 942 439 entries per year. Dividing it by 365 days, we get an average of 8061 entries per day. Dividing it once again by approximately 20 hours of operating time, we get an average of 403 entries per hour.

Following the peak hours, I will generate my data as follow:
- From 7:00 to 9:00 and from 16:00 to 18:00 (peak hours), the number of passengers will be a random number between 700 and 900.
- From 5:30 to 6:00 and from 23:00 to the closing time, the number of passengers will be a random number between 100 and 300.
- For the other hours, the number of passengers will be a random number between 300 and 500.
The average number of passengers for the day will be around 450, which is reasonable for the estimation we made above.


Additionnally, I will also assume that the number of passengers is constant throughout all 3 metro stations (I will assume that the number of passengers that are leaving the station is equal to the number of passengers that is entering the station; I am quite aware that this is not true, but I will assume this for now).

Lastly, the schedule of frequency for the metro towards St-Michel and the metro towards Snowdon will be the same (I'll allow this since the operating hours are the same for both ways).

In [ ]:
# NB: for simplicity reasons, the frequency of the metro per hour will be integers.

import pandas as pd
import numpy as np
from statistics import mean
from math import floor

def get_frequency(hour, type):
    time_count = 0
    freq_mean = 0
    freq_mean = []
    if type == "weekday":
      while time_count < 60:
        # Peak hours frequency
        if 7 <= hour < 9 or 16 <= hour < 18:
          time = np.random.randint(3, 5)
          time_count += time
          freq_mean.append(time)
        else: # Non-peak hours
          time = np.random.randint(4, 10)
          time_count += time
          freq_mean.append(time)
    else:
      time = np.random.randint(8, 11)
      time_count += time
      freq_mean.append(time)

    # Adjust the frequency depending on if it is a full 60 minutes or not
    if hour == 0 and (type == "weekday" or type == "sunday"):
      return floor(45 / mean(freq_mean))
    elif hour == 1 and type == "saturday":
      return floor(15 / mean(freq_mean))
    elif hour == 5:
      return floor(30 / mean(freq_mean))
    else:
      return floor(60 / mean(freq_mean))


# Random estimations. I found some estimations online, but they were not 100% reliable so I'll start with my own numbers.
# I will assume that very early during the day (5:30AM to 6AM) and towards midnight (11PM to closing time), the number of passengers per hour is less than during the non-peak hours.
def get_passengers(hour):
    if 7 <= hour <= 9 or 16 <= hour <= 18:
        return np.random.randint(700, 900)
    elif 5 <= hour <= 6 or 23 <= hour or 0 <= hour <= 1:
        return np.random.randint(100, 300)
    else:
        return np.random.randint(300, 500)

# Print the schedule
def generate_schedule(start_hour, start_minute, end_hour, end_minute, type):
    hours = []
    current_hour, current_minute = start_hour, start_minute
    iteration_count = 0

    while True:
        hours.append(f"{current_hour:02}:{current_minute:02}")

        if current_hour == end_hour and current_minute == end_minute:
            break

        current_hour += 1
        current_minute = 0

        if current_hour == 24:
          current_hour = 0

        iteration_count += 1
        if iteration_count > 24:
            print("Error.")
            break

    return pd.DataFrame({
        "Hour": hours,
        "Metro_Freq_per_Hour": [get_frequency(int(hour[:2]), type) for hour in hours],
        "Passengers_per_Hour": [get_passengers(int(hour[:2])) for hour in hours]
    })


# Since the ending hours are not round, for my schedule dataframe, I will simply write the Hour only and not the minutes for the ending hour.
# For example, on weekdays, the metro stops are 00:45, so my last row in the dataframe will be 00:00 but it will represent
# the frequency and the number of passengers from 00:00 to 00:45.

# Similarly, since the opening hour is 05:30 for everyday, the frequency and the number of passengers are for 5:30 to 6:00,
# which is why the hour in the first row in the first row of the dataframes starts at 5:00 and not 5:30.
weekday_df = generate_schedule(5, 30, 0, 0, "weekday")
saturday_df = generate_schedule(5, 30, 1, 0, "saturday")
sunday_df = generate_schedule(5, 30, 0, 0, "sunday")

print("Weekday Schedule:\n", weekday_df)
print("\nSaturday Schedule:\n", saturday_df)
print("\nSunday Schedule:\n", sunday_df)


Weekday Schedule:
      Hour  Metro_Freq_per_Hour  Passengers_per_Hour
0   05:30                    4                  177
1   06:00                    8                  219
2   07:00                   16                  797
3   08:00                   17                  798
4   09:00                    9                  781
5   10:00                    8                  411
6   11:00                    8                  475
7   12:00                    8                  444
8   13:00                   10                  371
9   14:00                    8                  391
10  15:00                    9                  351
11  16:00                   18                  753
12  17:00                   17                  851
13  18:00                    9                  844
14  19:00                    9                  415
15  20:00                    9                  355
16  21:00                    8                  428
17  22:00                    8               

Now, I need to generate the bank of employees where for each of them, we have [employee number, availability to work  [day type, hours]].

I will set some values arbitrarily. The range of hours to set an availability: 4 to 8 hours hours (for example, an employee cannot write [tuesday, 11-13] as an available window to work).

*NB: Hour 25 will represent 1AM.*

In [ ]:
# I'll impose that a shift can only be between 4 and 8 hours. 4 as minimum, 8 as maximum to not overexhaust the employees
def generate_employee_availability(num_employees):
    employee_bank = []
    for employee_id in range(1, num_employees + 1):
        availability = {"ID": employee_id}
        for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]:
            if np.random.choice([True, False]):
                start_hour = np.random.randint(5, 21)
                end_hour = 0
                while end_hour == 0 or end_hour > 25:
                    end_hour = np.random.randint(start_hour + 4, start_hour + 8)
                availability[day] = (start_hour, end_hour)
            else:
                availability[day] = (0,0)
        employee_bank.append(availability)
    return pd.DataFrame(employee_bank)

# Create a bank of 1500 employees; cannot a number to small could cause a problem when we'll try to schedule employees for the buses
employee_df = generate_employee_availability(2000)
print(employee_df)

        ID    Monday   Tuesday Wednesday  Thursday    Friday  Saturday  \
0        1  (18, 24)    (0, 0)    (0, 0)    (0, 0)  (11, 17)  (10, 16)   
1        2    (0, 0)    (0, 0)  (16, 21)  (16, 21)    (0, 0)  (11, 17)   
2        3    (0, 0)    (0, 0)    (0, 0)  (20, 25)    (0, 0)    (0, 0)   
3        4   (6, 10)  (14, 18)  (14, 21)    (0, 0)  (14, 19)   (5, 11)   
4        5    (0, 0)    (0, 0)    (0, 0)    (0, 0)    (0, 0)   (7, 13)   
...    ...       ...       ...       ...       ...       ...       ...   
1995  1996    (0, 0)  (16, 22)    (0, 0)  (13, 17)    (0, 0)  (14, 19)   
1996  1997    (0, 0)    (5, 9)  (14, 21)   (7, 14)  (13, 18)    (0, 0)   
1997  1998   (5, 12)  (13, 20)  (13, 20)  (20, 24)    (0, 0)    (0, 0)   
1998  1999  (13, 17)    (0, 0)  (11, 17)    (0, 0)  (15, 20)   (6, 13)   
1999  2000    (0, 0)    (0, 0)    (0, 0)   (5, 10)   (8, 12)  (18, 24)   

        Sunday  
0     (14, 20)  
1       (0, 0)  
2     (17, 23)  
3       (0, 0)  
4     (18, 23)  
...      

# Step 2: model the problem

First, here are important facts we need in our problem:
- Capacity of a metro's train: 1000 passengers/train
- Capacity of a small bus: 80 passengers
- Capacity of a big bus: 112 passengers
- Ratio of cost of using a small bus: 1
- Ratio of cost of using a big bus: 1.2


The **decision variables** are the following:
- $x_{small, h, d}$: number of small buses needed at hour $h$ for type $d$ of the day.
- $x_{big, h, d}$: number of big buses needed at hour $h$ for type $d$ of the day.
- $e_{i, h, d}$: employee $i$ on day $d$ for hour $h$. This variable is equal to $1$ is the employee is scheduled, $0$ if not.

The **parameters** are:
- $c_{small} = 80$ : capacity of a small bus.
- $c_{big} = 112$ : capacity of a big bus.
- $p_{h, d}$ : number of passengers at hour $h$ for day $d$.
- $h \in H_d$: operating hours of day $d$.
- $d \in D$: the types of days in a week.
- $i \in E$: employee $i$ in bank of employees $E$.

The **constraints** are: \\
- Demand constraint: $x_{small,h,d} * c_{small} + x_{big,h,d} * c_{big} \geq p_{h,d} ∀ h \in H_d$ \\
- Non-negativity constraints: $x_{small,h, d}, x_{big,h,d} \geq 0$ \\
- Integer contraints: $x_{small,h,d}, x_{big,h,d}$ are integers \\
- $e_{i, h, d} \in \{0,1\}$
- Employee for each bus constraint: $\sum\limits_{i \in E} e_{i,h,d} = x_{small,h,d} + x_{big,h,d} ∀ h \in H_d$
- If $e_{i,h,d} = 1$ for $i \in E$ and $e_{i-1,h,d} = 0$, we must have $e_{i,h+1,d}, e_{i,h+2,d}$ (minimum of 3-hour shifts).
- Restrict multiple shifts in a day:
  - Add a constraint so that the employee cannot be scheduled for the following hours.
   $\sum\limits_{h=i+1}^{n-1} e_{i, h} \leq (1 - shift\_end_{i, h}) * n$ for $h \in {0, 1, ..., n-1}$. Basically, if $shift\_end$ is equal to 1, so the employee's shift has ended, the right side of the inequality is 0 so an employee cannot be scheduled for the following hours. Instead, if $shift\_end$ is equal to 0, an employee can still be scheduled for the following hours.


The **objective** is to minimize the number of buses on the road for every hour. \\
$min \sum\limits_{h \in H_d, d \in D} x_{small,h,d} + x_{big,h,d}$


In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 73.3 MB/s eta 0:00:00


First, here would be the code if I only needed the optimal number of buses per hour for each day of the week (without the scheduled employees).

At first, my objective function was model += lpSum(x_small) + lpSum(x_big), but when defined like this, it was more optimal for my code to use bigger buses instead of small ones. To be able to get a mix of the two, I had to add the constraint that the cost of using a big bus is 1.2 bigger than the cost of using a small bus.

In [ ]:
import pandas as pd
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpInteger

c_small = 80
c_big = 112

types = [("Weekday", weekday_df), ("Saturday", saturday_df), ("Sunday", sunday_df)]

for day_name, t in types:
    model = LpProblem(f"{day_name}_Bus_Schedule_Optimization", LpMinimize)

    x_small = [LpVariable(f"small_buses_{day_name}_{i}", lowBound=0, cat=LpInteger) for i in range(len(t))]
    x_big = [LpVariable(f"big_buses_{day_name}_{i}", lowBound=0, cat=LpInteger) for i in range(len(t))]

    # Constraint of demand: make sure that the number of buses on the road will satisfy the demand of number of passengers
    for i, row in t.iterrows():
        passengers = row["Passengers_per_Hour"]
        model += x_small[i] * c_small + x_big[i] * c_big >= passengers

    model += lpSum(x_small) + 1.2*lpSum(x_big)

    model.solve()

    # Print the optimal number of buses
    print(f"{day_name} optimal number of buses:")
    for i, row in t.iterrows():
        print(f"Hour: {row['Hour']} --> {x_small[i].varValue} small buses and {x_big[i].varValue} big buses")
    print("\n")

Weekday optimal number of buses:
Hour: 05:30 --> 1.0 small buses and 1.0 big buses
Hour: 06:00 --> 0.0 small buses and 2.0 big buses
Hour: 07:00 --> 3.0 small buses and 5.0 big buses
Hour: 08:00 --> 3.0 small buses and 5.0 big buses
Hour: 09:00 --> 0.0 small buses and 7.0 big buses
Hour: 10:00 --> 1.0 small buses and 3.0 big buses
Hour: 11:00 --> 2.0 small buses and 3.0 big buses
Hour: 12:00 --> 0.0 small buses and 4.0 big buses
Hour: 13:00 --> 2.0 small buses and 2.0 big buses
Hour: 14:00 --> 1.0 small buses and 3.0 big buses
Hour: 15:00 --> 3.0 small buses and 1.0 big buses
Hour: 16:00 --> 0.0 small buses and 7.0 big buses
Hour: 17:00 --> 1.0 small buses and 7.0 big buses
Hour: 18:00 --> 1.0 small buses and 7.0 big buses
Hour: 19:00 --> 1.0 small buses and 3.0 big buses
Hour: 20:00 --> 2.0 small buses and 2.0 big buses
Hour: 21:00 --> 0.0 small buses and 4.0 big buses
Hour: 22:00 --> 0.0 small buses and 4.0 big buses
Hour: 23:00 --> 1.0 small buses and 1.0 big buses
Hour: 00:00 --> 2

In [ ]:
# If we want the time schedule of the buses
import pandas as pd
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpInteger

c_small = 80
c_big = 112

types = [("Weekday", weekday_df), ("Saturday", saturday_df), ("Sunday", sunday_df)]

for day_name, t in types:
    print(f"{day_name} optimal number of buses:")
    model = LpProblem(f"{day_name}_Bus_Schedule_Optimization", LpMinimize)

    x_small = [LpVariable(f"small_buses_{day_name}_{i}", lowBound=0, cat=LpInteger) for i in range(len(t))]
    x_big = [LpVariable(f"big_buses_{day_name}_{i}", lowBound=0, cat=LpInteger) for i in range(len(t))]

    for i, row in t.iterrows():
        passengers = row["Passengers_per_Hour"]
        model += x_small[i] * c_small + x_big[i] * c_big >= passengers

    model += lpSum(x_small) + 1.2 * lpSum(x_big)

    model.solve()

    for i, row in t.iterrows():
        hour, minute = map(int, row['Hour'].split(":"))

        total_buses = int(x_small[i].varValue) + int(x_big[i].varValue)
        if total_buses > 0:
            if hour == 5:
              interval = 30 // total_buses
            elif hour == 1:
              interval = 15 // total_buses
            else:
              interval = 60 // total_buses
        else:
            interval = 60

        departure_times = []
        for j in range(total_buses):
            if hour == 5:
              departure_times.append(f"{hour:02}:{30 + j * interval:02}")
            else:
              departure_times.append(f"{hour:02}:{minute + j * interval:02}")

        # Print the schedule for the current hour
        print(f"Hour: {row['Hour']} --> {x_small[i].varValue} small buses and {x_big[i].varValue} big buses --> {', '.join(departure_times)}")
    print("\n")

Weekday optimal number of buses:
Hour: 05:30 --> 1.0 small buses and 1.0 big buses --> 05:30, 05:45
Hour: 06:00 --> 0.0 small buses and 2.0 big buses --> 06:00, 06:30
Hour: 07:00 --> 0.0 small buses and 8.0 big buses --> 07:00, 07:07, 07:14, 07:21, 07:28, 07:35, 07:42, 07:49
Hour: 08:00 --> 2.0 small buses and 6.0 big buses --> 08:00, 08:07, 08:14, 08:21, 08:28, 08:35, 08:42, 08:49
Hour: 09:00 --> 0.0 small buses and 8.0 big buses --> 09:00, 09:07, 09:14, 09:21, 09:28, 09:35, 09:42, 09:49
Hour: 10:00 --> 3.0 small buses and 2.0 big buses --> 10:00, 10:12, 10:24, 10:36, 10:48
Hour: 11:00 --> 2.0 small buses and 3.0 big buses --> 11:00, 11:12, 11:24, 11:36, 11:48
Hour: 12:00 --> 0.0 small buses and 3.0 big buses --> 12:00, 12:20, 12:40
Hour: 13:00 --> 1.0 small buses and 3.0 big buses --> 13:00, 13:15, 13:30, 13:45
Hour: 14:00 --> 2.0 small buses and 2.0 big buses --> 14:00, 14:15, 14:30, 14:45
Hour: 15:00 --> 1.0 small buses and 3.0 big buses --> 15:00, 15:15, 15:30, 15:45
Hour: 16:00 -

As we can see, right now, the optimal solution uses a mix of both small buses and big buses.

Now, in the code following, I add the aspect of the employees.

I tried to implement something but it didn't entirely work. I wanted to impose that when an employee is scheduled, his/her shift should be a minimum of 3 hours. This is too avoid having different employees for every hour of the day. It would also be annoying for the employees to only work 1 hour per day.

In [ ]:
import pandas as pd
import numpy as np
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpInteger

c_small = 80
c_big = 112
types = [("Monday", weekday_df), ("Tuesday", weekday_df), ("Wednesday", weekday_df), ("Thursday", weekday_df), ("Friday", weekday_df), ("Saturday", saturday_df), ("Sunday", sunday_df)]
weekly_schedule = {day_name: [] for day_name, _ in types}


for day_name, schedule in types:
    model = LpProblem(f"{day_name}_Bus_Schedule_Optimization", LpMinimize)

    # Decision variables for the buses
    x_small = [LpVariable(f"small_buses_{day_name}_{i}", lowBound=0, cat=LpInteger) for i in range(len(schedule))]
    x_big = [LpVariable(f"big_buses_{day_name}_{i}", lowBound=0, cat=LpInteger) for i in range(len(schedule))]

    employees = employee_df["ID"].tolist()
    e = {(emp, hour): LpVariable(f"employee_{emp}_hour_{hour}", cat="Binary")
         for emp in employees for hour in range(len(schedule))}

    shift_end = {(emp, hour): LpVariable(f"shift_end_{emp}_hour_{hour}", cat="Binary")
                  for emp in employees for hour in range(len(schedule))}

    # Constraint of demand: make sure there are enough buses on the road to respect the demand
    for i, row in schedule.iterrows():
        passengers = row["Passengers_per_Hour"]
        model += x_small[i] * c_small + x_big[i] * c_big >= passengers
        model += lpSum(e[(emp, i)] for emp in employees) == x_small[i] + x_big[i]

    for emp in employees:
        availability = employee_df.loc[employee_df["ID"] == emp]
        for i, row in schedule.iterrows():
            hour = int(row["Hour"].split(":")[0])
            day = day_name

            # Verify if the employee is available for a 3-hour shift
            available_for_3_hours = True
            for j in range(3):
                if i + j >= len(schedule):  # Verify if the day would be over of not if the employee works 3 hours
                    available_for_3_hours = False
                    break
                next_hour = int(schedule.iloc[i + j]["Hour"].split(":")[0])
                # Verify is the employee is actually available or not. ex: next_hour is 15. availability[day].iloc[0][0] and availability[day].iloc[0][1]
                # are taking the availability of the employee, let's say its 12-20. then since 12 <= 15 <= 20, available_for_3_hours stays true.
                # Instead, if the next_hour was not constrained in the availability, the variable becomes false.
                if day not in availability or pd.isna(availability[day].iloc[0]) or not (availability[day].iloc[0][0] <= next_hour <= availability[day].iloc[0][1]):
                    available_for_3_hours = False
                    break

            if available_for_3_hours:  # Employee is available for the next 3 hours
                # Check if this is the start of a shift
                is_shift_start = True
                if i > 0:  # Not the first hour of the day
                   is_shift_start = (e[(emp, i - 1)].varValue == 0) and (e[(emp, i)].varValue == 1)
                if is_shift_start: # Schedule the employee for the next two hours
                    model += e[(emp, i)] == e[(emp, i + 1)]
                    model += e[(emp, i)] == e[(emp, i + 2)]
            else:  # Not available for 3 hours
                if i + 2 < len(schedule):
                    model += e[(emp, i)] == 0

        # Constraint so that an employee cannot work 2 or more shifts in a day (ex: cannot work hour 14,15 and 20,21,22)
        for i in range(len(schedule)):
            if i < len(schedule) - 1:
                model += e[(emp, i)] - e[(emp, i + 1)] <= shift_end[(emp, i)]
            else:
                model += e[(emp, i)] <= shift_end[(emp, i)]
            model += lpSum(e[(emp, j)] for j in range(i + 1, len(schedule))) <= (1 - shift_end[(emp, i)]) * len(schedule)


    model += lpSum(x_small) + 1.3 * lpSum(x_big)

    model.solve()

    # Print the whole schedule
    print(f"{day_name} optimal number of buses and employee schedule:")
    for i, row in schedule.iterrows():
        hour = int(row["Hour"].split(":")[0])
        small_buses = x_small[i].varValue
        big_buses = x_big[i].varValue
        assigned_employees = [emp for emp in employees if e[(emp, i)].varValue == 1]
        weekly_schedule[day_name].append((hour, small_buses, big_buses, assigned_employees))
        print(f"Hour: {row['Hour']} --> {int(small_buses)} small buses, {int(big_buses)} big buses")
        print(f"   Assigned employees: {assigned_employees}")
    print("\n")


Monday optimal number of buses and employee schedule:
Hour: 05:30 --> 0 small buses, 1 big buses
   Assigned employees: [1012]
Hour: 06:00 --> 0 small buses, 1 big buses
   Assigned employees: [1012]
Hour: 07:00 --> 0 small buses, 7 big buses
   Assigned employees: [974, 1012, 1461, 1557, 1923]
Hour: 08:00 --> 0 small buses, 7 big buses
   Assigned employees: [974, 1012, 1461, 1557, 1565, 1923]
Hour: 09:00 --> 0 small buses, 6 big buses
   Assigned employees: [974, 1364, 1389, 1461, 1557, 1565]
Hour: 10:00 --> 0 small buses, 3 big buses
   Assigned employees: [741, 1364, 1389]
Hour: 11:00 --> 0 small buses, 4 big buses
   Assigned employees: [741, 1049, 1364]
Hour: 12:00 --> 0 small buses, 3 big buses
   Assigned employees: [741, 1049, 1364]
Hour: 13:00 --> 0 small buses, 3 big buses
   Assigned employees: [741, 814]
Hour: 14:00 --> 0 small buses, 3 big buses
   Assigned employees: [814, 1008, 1789]
Hour: 15:00 --> 0 small buses, 3 big buses
   Assigned employees: [1008, 1789]
Hour: 16

I thought it would be more intuitive to see this schedule in a table format.

*P.S.: I was so impressed by Google Colab. It gives us the possibility to convert the dataframe into an interactive table!!! And we can filter the data by the following aspects: index, day, hour, small buses, big buses, and assigned employees.*


In [ ]:
def create_schedule_table(weekly_schedule):
    data = []
    for day, slots in weekly_schedule.items():
        for hour, small, big, employees in slots:
            data.append([day, hour, int(small), int(big), ', '.join(map(str, employees))])

    df = pd.DataFrame(data, columns=['Day', 'Hour', 'Small Buses', 'Big Buses', 'Assigned Employees'])
    return df

schedule_df = create_schedule_table(weekly_schedule)

display(schedule_df)

,Day,Hour,Small Buses,Big Buses,Assigned Employees
0,Monday,5,0,1,1214
1,Monday,6,0,1,1214
2,Monday,7,0,7,"478, 1010, 1035, 1214, 1531, 1573, 1596"
3,Monday,8,0,7,"454, 1035, 1214, 1330, 1531, 1573, 1793"
4,Monday,9,0,7,"1011, 1035, 1214, 1330, 1484, 1531, 1793"
...,...,...,...,...,...
136,Sunday,20,0,4,"567, 1145, 1302"
137,Sunday,21,0,3,"567, 1145, 1302"
138,Sunday,22,0,3,"1302, 1613"
139,Sunday,23,0,2,1302


I also thought that it would be fun if employees could quickly know their schedules for the week by only using their employee number. The question will loop in case we want to check for multiple employees.

In [ ]:
import pandas as pd

# To show the times with the right minutes and to respect the ending operating hours depending on the day
def format_hour(hour, type):
    if hour == 5:
        return "05:30"
    elif hour == 0 and (type == "monday" or type == "tuesday" or type == "wednesday" or type == "thursday" or type == "friday" or type == "sunday"):
        return "00:45"
    elif hour == 1 and type == "saturday":
        return "01:15"
    else:
        return f"{hour:02}:00"

def get_employee_schedule(employee_id, weekly_schedule, employee_df):
    schedule = {}
    for day, slots in weekly_schedule.items():
        day_type = day.lower()
        availability = employee_df.loc[employee_df["ID"] == employee_id, day]
        schedule[day] = []
        if not availability.empty and pd.notna(availability.iloc[0]):
            start_hour, end_hour = availability.iloc[0]
            work_slots = [
                format_hour(hour, day_type)
                for hour, small, big, employees in slots if employee_id in employees
            ]
            schedule[day] = work_slots
    return schedule

while True:
    try:
        employee_id = int(input("Enter employee ID: "))
        emp_schedule = get_employee_schedule(employee_id, weekly_schedule, employee_df)
        print(f"Weekly schedule for employee {employee_id}:")
        for day, slots in emp_schedule.items():
            print(f"{day}: {slots}")

        another_employee = input("\nGet schedule for another employee? (yes/no): ")
        if another_employee.lower() != "yes":
            break
    except ValueError:
        print("Invalid employee ID. Please enter a number.")

Enter employee ID: 1494
Weekly schedule for employee 1494:
Monday: ['07:00', '08:00', '09:00', '10:00']
Tuesday: []
Wednesday: []
Thursday: []
Friday: []
Saturday: []
Sunday: []

Get schedule for another employee? (yes/no): yes
Enter employee ID: 434
Weekly schedule for employee 434:
Monday: ['13:00', '15:00']
Tuesday: []
Wednesday: []
Thursday: ['16:00']
Friday: []
Saturday: []
Sunday: []

Get schedule for another employee? (yes/no): yes
Enter employee ID: 10
Weekly schedule for employee 10:
Monday: []
Tuesday: []
Wednesday: []
Thursday: []
Friday: []
Saturday: []
Sunday: []

Get schedule for another employee? (yes/no): no


# Bibliography

Not a full biography, but here are the links to the information I found about the STM.

- Opening hours and frequency of departure of the Blue Line: https://www.stm.info/en/info/networks/metro/ligne-5---bleue
- Average passengers per day: https://en.wikipedia.org/wiki/Montreal_Metro#Lines_and_operation
- Rapport d'activité (l'année en chiffres): https://www.stm.info/fr/a-propos/informations-entreprise-et-financieres/rapport-annuel-2023/rapport-dactivite
- Metro capacity: https://www.stm.info/en/press/news/2018/major-milestone-for-montreal-metro--10-billion-boardings-since-system-opened--french-only-#:~:text=During%20rush%20hours%2C%2080%20m%C3%A9tro,comprises%2071%20kilometres%20of%20tunnels.
- Bus capacity (small and big buses): https://www.stm.info/sites/default/files/affairespubliques/Communiques/Annexes/annexe_2_fiche_tech._lfs_nova.pdf